In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.tensorboard.writer import SummaryWriter

from sklearn.model_selection import train_test_split

from common import data
from common.training import training_loop

from algos import deep_fib

from common.models import resnet, deeplab, scinet

SCINET = "scinet"
DEEPLAB = "deeplab"

In [3]:
paths = data.get_dataset_paths("../data")
train, test = train_test_split(paths, test_size=0.1, random_state=42)


In [4]:
m_data_train = data.Marconi100Dataset(train, scaling=data.Scaling.MINMAX)
m_data_test = data.Marconi100Dataset(test, scaling=data.Scaling.MINMAX)


Loading:  57%|█████▋    | 128/224 [04:53<04:14,  2.65s/it]

In [ ]:
model_name = DEEPLAB
horizon = 1024
stride = 512
n_masks = 50

batch_size = 32

log_dir = "./outputs/deep_fib_deeplab"
lr = 1e-3
num_epochs = 30

anomaly_threshold = 0.1  # to be tuned

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
dataset_train = data.UnfoldedDataset(m_data_train, horizon=horizon, stride=stride)
dataset_test = data.UnfoldedDataset(m_data_test, horizon=horizon, stride=stride)

masks = deep_fib.get_masks(horizon, n_masks).float()

print(len(dataset_train), len(dataset_test), masks.size())


In [ ]:
train_loader = DataLoader(
    dataset_train,
    batch_size,
    shuffle=True,
)
test_loader = DataLoader(
    dataset_test,
    batch_size,
    shuffle=False,
)
print(len(train_loader), len(test_loader))


In [ ]:
if model_name == SCINET:
    num_encoder_levels = 3
    hidden = [512]
    input_dim = 460
    hidden_size = 2
    kernel_size = 3
    dropout = 0.5

    model = scinet.SCINet(
        output_len=horizon,
        input_len=horizon,
        num_encoder_levels=num_encoder_levels,
        hidden_decoder_sizes=hidden,
        input_dim=input_dim,
        hidden_size=hidden_size,
        kernel_size=kernel_size,
        dropout=dropout,
    ).float()
    with open(f"{log_dir}/config.json", "w") as f:
        json.dump(
            dict(
                output_len=horizon,
                input_len=horizon,
                num_encoder_levels=num_encoder_levels,
                hidden_decoder_sizes=hidden,
                input_dim=input_dim,
                hidden_size=hidden_size,
                kernel_size=kernel_size,
                dropout=dropout,
            ),
            f,
        )
elif model_name == DEEPLAB:
    model = deeplab.DeepLabNet(
        resnet.ResNetFeatures(
            resnet.Bottleneck,
            resnet.RESNET50_LAYERS,
            return_layers=[resnet.LAYER_1, resnet.LAYER_4],
            replace_stride_with_dilation=[False, True, True],
            num_features=data.NUM_FEATURES,
        ),
        backbone_channels=[256, 2048],
        out_feats=data.NUM_FEATURES,
    ).float()

In [ ]:
engine = deep_fib.DeepFIBEngine(anomaly_threshold, masks)
optim = Adam(model.parameters(), lr=lr)
lr_sched = CosineAnnealingLR(optim, num_epochs)

with SummaryWriter(log_dir) as writer:
    training_loop(
        model=model,
        engine=engine,
        num_epochs=num_epochs,
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        device=device,
        optimizer=optim,
        lr_scheduler=lr_sched,
        writer=writer,
        save_path=log_dir + "/models",
    )